# GEM-PRO - List of Gene IDs

This notebook gives an example of how to run the GEM-PRO pipeline with a **list of gene IDs**.
<div class="alert alert-info">

**Input:** List of gene IDs

</div>

<div class="alert alert-info">

**Output:** GEM-PRO model

</div>


## Imports

In [1]:
import sys
import logging

In [2]:
# Import the GEM-PRO class
from ssbio.pipeline.gempro import GEMPRO

In [3]:
# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Logging

Set the logging level in `logger.setLevel(logging.<LEVEL_HERE>)` to specify how verbose you want the pipeline to be. Debug is most verbose.

- `CRITICAL`
     - Only really important messages shown
- `ERROR`
     - Major errors
- `WARNING`
     - Warnings that don't affect running of the pipeline
- `INFO` (default)
     - Info such as the number of structures mapped per gene
- `DEBUG`
     - Really detailed information that will print out a lot of stuff
     
<div class="alert alert-warning">

**Warning:** 
`DEBUG` mode prints out a large amount of information, especially if you have a lot of genes. This may stall your notebook!
</div>

In [4]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # SET YOUR LOGGING LEVEL HERE #

In [5]:
# Other logger stuff for Jupyter notebooks
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] %(levelname)s: %(message)s', datefmt="%Y-%m-%d %H:%M")
handler.setFormatter(formatter)
logger.handlers = [handler]

## Initialization of the project

Set these three things:

- `ROOT_DIR`
    - The directory where a folder named after your `PROJECT` will be created
- `PROJECT`
    - Your project name
- `LIST_OF_GENES`
    - Your list of gene IDs
    
A directory will be created in `ROOT_DIR` with your `PROJECT` name. The folders are organized like so:
```
    ROOT_DIR
    └── PROJECT
        ├── data  # General storage for pipeline outputs
        ├── model  # SBML and GEM-PRO models are stored here
        ├── genes  # Per gene information
        │   ├── <gene_id1>  # Specific gene directory
        │   │   └── protein
        │   │       ├── sequences  # Protein sequence files, alignments, etc.
        │   │       └── structures  # Protein structure files, calculations, etc.
        │   └── <gene_id2>
        │       └── protein
        │           ├── sequences
        │           └── structures
        ├── reactions  # Per reaction information
        │   └── <reaction_id1>  # Specific reaction directory
        │       └── complex
        │           └── structures  # Protein complex files
        └── metabolites  # Per metabolite information
            └── <metabolite_id1>  # Specific metabolite directory
                └── chemical
                    └── structures  # Metabolite 2D and 3D structure files
                
```

<div class="alert alert-info">**Note:** Methods for protein complexes and metabolites are still in development.</div>

In [6]:
# SET FOLDERS AND DATA HERE
import tempfile
ROOT_DIR = tempfile.gettempdir()

PROJECT = 'genes_GP'
LIST_OF_GENES = ['b0761', 'b0889', 'b0995', 'b1013', 'b1014', 'b1040', 'b1130', 'b1187', 'b1221', 'b1299']

In [7]:
# Create the GEM-PRO project
my_gempro = GEMPRO(gem_name=PROJECT, root_dir=ROOT_DIR, genes_list=LIST_OF_GENES)

[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: /tmp/genes_GP: GEM-PRO project location
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: 10: number of genes


## Mapping gene ID --> sequence

First, we need to map these IDs to their protein sequences. There are 2 ID mapping services provided to do this - through **KEGG** or **UniProt**. The end goal is to map a UniProt ID to each ID, since there is a comprehensive mapping (and some useful APIs) between UniProt and the PDB.

<p><div class="alert alert-info">**Note:** You only need to map gene IDs using one service. However you can run both if some genes don't map in one service and do map in another!</div></p>

### Methods

In [8]:
# KEGG mapping of gene ids
my_gempro.kegg_mapping_and_metadata(kegg_organism_code='eco')
print('Missing KEGG mapping: ', my_gempro.missing_kegg_mapping)
my_gempro.df_kegg_metadata.head()

[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: 10/10: number of genes mapped to KEGG
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: Completed ID mapping --> KEGG. See the "df_kegg_metadata" attribute for a summary dataframe.



Missing KEGG mapping:  []


,kegg,refseq,uniprot,num_pdbs,pdbs,seq_len,sequence_file,metadata_file
gene,,,,,,,,
b1130,eco:b1130,NP_415648,P23836,2,2PL1;2PKX,223,eco-b1130.faa,eco-b1130.kegg
b0889,eco:b0889,NP_415409,P0ACJ0,2,2GQQ;2L4A,164,eco-b0889.faa,eco-b0889.kegg
b1014,eco:b1014,NP_415534,P09546,16,3E2Q;4JNZ;3E2R;4JNY;2GPE;4O8A;3E2S;2FZN;1TJ1;1...,1320,eco-b1014.faa,eco-b1014.kegg
b1299,eco:b1299,NP_415815,P0A9U6,0,NaN,185,eco-b1299.faa,eco-b1299.kegg
b0761,eco:b0761,NP_415282,P0A9G8,5,1B9M;1H9S;1B9N;1O7L;1H9R,262,eco-b0761.faa,eco-b0761.kegg


In [9]:
# UniProt mapping
my_gempro.uniprot_mapping_and_metadata(model_gene_source='ENSEMBLGENOME_ID')
print('Missing UniProt mapping: ', my_gempro.missing_uniprot_mapping)
my_gempro.df_uniprot_metadata.head()

[2017-03-08 13:03] [root] INFO: getUserAgent: Begin
[2017-03-08 13:03] [root] INFO: getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.2; Linux) Python-requests/2.12.4
[2017-03-08 13:03] [root] INFO: getUserAgent: End
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: 10/10: number of genes mapped to UniProt
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: Completed ID mapping --> UniProt. See the "df_uniprot_metadata" attribute for a summary dataframe.



Missing UniProt mapping:  []


,uniprot,reviewed,gene_name,kegg,refseq,num_pdbs,pdbs,ec_number,pfam,seq_len,description,entry_version,seq_version,sequence_file,metadata_file
gene,,,,,,,,,,,,,,,
b1130,P23836,True,phoP,ecj:JW1116;eco:b1130,NP_415648.1;NC_000913.3;WP_001265471.1;NZ_LN83...,2,2PL1;2PKX,NaN,PF00072;PF00486,223,Transcriptional regulatory protein PhoP,2017-02-15,1991-11-01,P23836.fasta,P23836.txt
b0889,P0ACJ0,True,lrp,ecj:JW0872;eco:b0889,NP_415409.1;NC_000913.3;WP_000228473.1;NZ_LN83...,2,2L4A;2GQQ,NaN,PF01037,164,Leucine-responsive regulatory protein,2017-02-15,2007-01-23,P0ACJ0.fasta,P0ACJ0.txt
b1014,P09546,True,putA,ecj:JW0999;eco:b1014,NP_415534.1;NC_000913.3;WP_001326840.1;NZ_LN83...,16,4JNZ;2AY0;3E2S;4O8A;3E2R;3ITG;4JNY;3E2Q;2GPE;2...,1.2.1.88;1.5.5.2,PF00171;PF01619;PF14850,1320,L-glutamate gamma-semialdehyde dehydrogenase;P...,2017-02-15,1997-11-01,P09546.fasta,P09546.txt
b1299,P0A9U6,True,puuR,ecj:JW1292;eco:b1299,NP_415815.1;NC_000913.3;WP_001278727.1;NZ_LN83...,0,NaN,NaN,PF07883;PF01381,185,HTH-type transcriptional regulator PuuR,2017-02-15,2005-07-19,P0A9U6.fasta,P0A9U6.txt
b0761,P0A9G8,True,modE,ecj:JW0744;eco:b0761,NP_415282.1;NC_000913.3;WP_001147439.1;NZ_LN83...,5,1B9N;1O7L;1H9S;1B9M;1H9R,NaN,PF00126;PF03459,262,Transcriptional regulator ModE,2017-02-15,2005-07-19,P0A9G8.fasta,P0A9G8.txt


In [10]:
# Set representative sequences
my_gempro.set_representative_sequence()
print('Missing a representative sequence: ', my_gempro.missing_representative_sequence)
my_gempro.df_representative_sequences.head()

[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: 10/10: number of genes with a representative sequence
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: See the "df_representative_sequences" attribute for a summary dataframe.



Missing a representative sequence:  []


,uniprot,kegg,num_pdbs,pdbs,seq_len,sequence_file
gene,,,,,,
b1130,P23836,ecj:JW1116;eco:b1130,2,2PL1;2PKX,223,P23836.fasta
b0889,P0ACJ0,ecj:JW0872;eco:b0889,2,2L4A;2GQQ,164,P0ACJ0.fasta
b1014,P09546,ecj:JW0999;eco:b1014,16,4JNZ;2AY0;3E2S;4O8A;3E2R;3ITG;4JNY;3E2Q;2GPE;2...,1320,P09546.fasta
b1299,P0A9U6,ecj:JW1292;eco:b1299,0,NaN,185,P0A9U6.fasta
b0761,P0A9G8,ecj:JW0744;eco:b0761,5,1B9N;1O7L;1H9S;1B9M;1H9R,262,P0A9G8.fasta


## Mapping representative sequence --> structure

These are the ways to map sequence to structure:

1. Use the UniProt ID and their automatic mappings to the PDB
2. BLAST the sequence to the PDB
3. Make homology models or 
4. Map to existing homology models

You can only utilize option #1 to map to PDBs if there is a mapped UniProt ID set in the representative sequence. If not, you'll have to BLAST your sequence to the PDB or make a homology model. You can also run both for maximum coverage.

### Methods

In [11]:
# Mapping using the PDBe best_structures service
my_gempro.map_uniprot_to_pdb(seq_ident_cutoff=.3)
my_gempro.df_pdb_ranking.head()

[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: Mapping UniProt IDs --> PDB IDs...
[2017-03-08 13:03] [root] INFO: getUserAgent: Begin
[2017-03-08 13:03] [root] INFO: getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.2; Linux) Python-requests/2.12.4
[2017-03-08 13:03] [root] INFO: getUserAgent: End
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: 8/10: number of genes with at least one experimental structure
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: Completed UniProt --> best PDB mapping. See the "df_pdb_ranking" attribute for a summary dataframe.


,pdb_id,pdb_chain_id,uniprot,experimental_method,resolution,coverage,start,end,unp_start,unp_end,rank
gene,,,,,,,,,,,
b1130,2pl1,A,P23836,X-ray diffraction,1.90,0.543,1,121,1,121,1
b1130,2pkx,A,P23836,X-ray diffraction,2.54,0.543,1,121,1,121,2
b1130,2pkx,B,P23836,X-ray diffraction,2.54,0.543,1,121,1,121,3
b0889,2gqq,A,P0ACJ0,X-ray diffraction,3.20,0.994,1,163,2,164,1
b0889,2gqq,B,P0ACJ0,X-ray diffraction,3.20,0.994,1,163,2,164,2


In [12]:
# Mapping using BLAST
my_gempro.blast_seqs_to_pdb(all_genes=True, seq_ident_cutoff=.9, evalue=0.00001)
my_gempro.df_pdb_blast.head(2)

[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: Completed sequence --> PDB BLAST. See the "df_pdb_blast" attribute for a summary dataframe.
[2017-03-08 13:03] [ssbio.pipeline.gempro] INFO: 1: number of genes with additional structures added from BLAST


,pdb_id,pdb_chain_id,hit_score,hit_evalue,hit_percent_similar,hit_percent_ident,hit_num_ident,hit_num_similar
gene,,,,,,,,
b1013,4x1e,A,966.0,1.283880e-104,0.910377,0.910377,193,193
b1013,4x1e,B,966.0,1.283880e-104,0.910377,0.910377,193,193


## Downloading and ranking structures

### Methods

<div class="alert alert-warning">

**Warning:** 
Downloading all PDBs takes a while, since they are also parsed for metadata. You can skip this step and just set representative structures below if you want to minimize the number of PDBs downloaded.

</div>

In [13]:
# Download all mapped PDBs and gather the metadata
my_gempro.pdb_downloader_and_metadata()
my_gempro.df_pdb_metadata.head(2)

[2017-03-08 13:04] [ssbio.pipeline.gempro] INFO: Updated PDB metadata dataframe. See the "df_pdb_metadata" attribute for a summary dataframe.
[2017-03-08 13:04] [ssbio.pipeline.gempro] INFO: Saved 40 structures total


,chemicals,date,description,experimental_method,mapped_chains,pdb_id,pdb_title,resolution,structure_file,taxonomy_name
gene,,,,,,,,,,
b1130,PT;MG;BEF,2007-05-22;2007-08-14;2009-02-24,Transcriptional regulatory protein phoP,X-ray diffraction,A,2pl1,Berrylium Fluoride activated receiver domain o...,1.90,2pl1.cif,Escherichia coli
b1130,NaN,2007-05-22;2007-08-14;2009-02-24,Transcriptional regulatory protein phoP,X-ray diffraction,A;B,2pkx,E.coli response regulator PhoP receiver domain,2.54,2pkx.cif,Escherichia coli


In [14]:
# Set representative structures
my_gempro.set_representative_structure()
my_gempro.df_representative_structures.head()

[2017-03-08 13:04] [ssbio.core.protein] WARNING: b1130: no structures meet quality checks
[2017-03-08 13:04] [ssbio.core.protein] WARNING: b1014: no structures meet quality checks
[2017-03-08 13:05] [ssbio.core.protein] WARNING: b0995: no structures meet quality checks
[2017-03-08 13:05] [ssbio.pipeline.gempro] INFO: 5/10: number of genes with a representative structure
[2017-03-08 13:05] [ssbio.pipeline.gempro] INFO: See the "df_representative_structures" attribute for a summary dataframe.


,id,is_experimental,reference_seq,reference_seq_top_coverage
gene,,,,
b0889,2gqq-A,True,P0ACJ0,93.3
b0761,1b9m-A,True,P0A9G8,96.2
b1221,1a04-A,True,P0AF28,94.9
b1013,4jyk-A,True,P0ACU2,94.8
b1187,1hw1-A,True,P0A8V6,94.6


In [15]:
# Looking at the information saved within a gene
my_gempro.genes.get_by_id('b1187').protein.representative_structure
my_gempro.genes.get_by_id('b1187').protein.representative_structure.get_dict()

<StructProp 1hw1-A at 0x7f794cd88278>

{'_structure_dir': '/tmp/genes_GP/genes/b1187/b1187_protein/structures',
 'chains': [<ChainProp A at 0x7f79452bfb70>],
 'date': ['2001-01-24', '2001-06-06', '2003-04-01', '2009-02-24'],
 'description': 'FATTY ACID METABOLISM REGULATOR PROTEIN',
 'file_type': 'pdb',
 'id': '1hw1-A',
 'is_experimental': True,
 'mapped_chains': ['A'],
 'original_pdb_id': '1hw1',
 'reference_seq': <SeqProp P0A8V6 at 0x7f79452beb70>,
 'reference_seq_top_coverage': 94.6,
 'representative_chain': <ChainProp A at 0x7f79452be7f0>,
 'resolution': 1.5,
 'structure_file': '1hw1-A_clean.pdb',
 'taxonomy_name': 'Escherichia coli'}

## Saving your GEM-PRO

<p><div class="alert alert-warning">**Warning:** Saving is still experimental. For a full GEM-PRO with sequences & structures, depending on the number of genes, saving can take >5 minutes.</div></p>

In [16]:
import os.path as op
my_gempro.save_json(op.join(my_gempro.model_dir, '{}.json'.format(my_gempro.id)), compression=False)

[2017-03-08 13:05] [root] WARNING: json-tricks: numpy scalar serialization is experimental and may work differently in future versions
[2017-03-08 13:05] [ssbio.core.io] INFO: Saved <class 'ssbio.pipeline.gempro.GEMPRO'> (id: genes_GP) to /tmp/genes_GP/model/genes_GP.json
